In [84]:
from pathlib import Path
from itertools import product

import numpy as np
import healpy as hp
from sklearn.decomposition import PCA
from astropy.io import fits

import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

from tqdm import tqdm  # For progress bars

from cmbml.utils.handle_data import get_map_dtype, get_planck_obs_data, get_planck_noise_data

In [85]:
import logging

In [86]:
logger = logging.getLogger("handle_data")
logger.setLevel(logging.DEBUG)

In [87]:
DATA_ROOT = "/data/jim/CMB_Data/"
ASSETS_DIRECTORY = f"{DATA_ROOT}/Assets/Planck/"
PLANCK_NOISE_DIR = f"{DATA_ROOT}/Planck_Noise/"

DETECTORS = [545]
# DETECTORS = [30, 44, 70, 100, 143, 217, 353, 545, 857]
N_PLANCK_SIMS = 20

In [88]:
def get_lmax_for_nside(nside):
    """Helper function: Max ell for a given nside; to be considered a parameter"""
    return 3 * nside - 1

# Setup

# Planck Sims

In [89]:
def get_field_unit(fits_fn, hdu, field_idx):
    """
    Get the unit associated with a specific field from the header of the 
    specified HDU (Header Data Unit) in a FITS file.

    Args:
        fits_fn (str): The filename of the FITS file.
        hdu (int): The index of the HDU.
        field_idx (int): The index of the field.

    Returns:
        str: The unit of the field.
    """
    with fits.open(fits_fn) as hdul:
        try:
            field_num = field_idx + 1
            unit = hdul[hdu].header[f"TUNIT{field_num}"]
        except KeyError:
            unit = ""
    return unit

In [90]:
for det in DETECTORS:
    # Get a map filename
    t_fn = get_planck_noise_data(detector=det, assets_directory=ASSETS_DIRECTORY, realization=0, progress=True)
    # get the map unit
    unit = get_field_unit(t_fn, 1, 0)
    print(unit)

MJy/sr


In [91]:
def get_ps_data(detector):
    if detector in [30, 44, 70]:
        nside = 1024
    else:
        nside = 2048
    lmax = get_lmax_for_nside(nside)  # Defined above as 3*Nside-1
    # Getting power spectra for 100 maps at 100 GHz takes ~50 minutes
    src_cls = []
    maps_means = []
    for i in tqdm(range(N_PLANCK_SIMS)):
        src_map_fn = get_planck_noise_data(detector=detector, assets_directory=ASSETS_DIRECTORY, realization=i, progress=True)

        # Don't bother using astropy units here
        src_map_unit = get_field_unit(src_map_fn, 1, 0)
        if src_map_unit not in ["K_CMB", "MJy/sr"]:
            raise ValueError(f"Unknown unit {src_map_unit} for map {src_map_fn}")
        t_src_map = hp.read_map(src_map_fn)

        maps_means.append(np.mean(t_src_map))
        src_cls.append(hp.anafast(t_src_map, lmax=lmax))

    # Determine parameters for approximating the distribution of power spectra
    # Use log scaling for the power spectra; otherwise it's dominated by low ells
    log_src_cls = np.log10(src_cls)

    # We want to find the components that explain the majority of the variance
    #   We don't have enough maps to fully determine the distribution, but a full
    #   covariance matrix is overkill anyways. PCA gives a good, concise summary.
    pca = PCA().fit(log_src_cls)

    # We need the mean, the components (eigenvectors), and the variance (eigenvalues)
    #   These are surrogates for the full covariance matrix
    mean_ps = pca.mean_
    components = pca.components_  
    variance = pca.explained_variance_

    # We need the mean and standard deviation of the maps_means so we can adjust the monopole as needed
    maps_mean = np.mean(maps_means)
    maps_sd = np.std(maps_means)

    # Save the results; delete the variables so we know we test loading them
    np.savez(f"noise_model_{detector}GHz_n{N_PLANCK_SIMS}_uK.npz", 
             mean_ps=mean_ps, 
             components=components,
             variance=variance, 
             maps_mean=maps_mean, 
             maps_sd=maps_sd,
             map_unit=src_map_unit)

In [92]:
for det in DETECTORS:
    get_ps_data(detector=det)

100%|██████████| 20/20 [09:20<00:00, 28.01s/it]


In [93]:
for det in DETECTORS:
    data = np.load(f"noise_model_{det}GHz_n{N_PLANCK_SIMS}_uK.npz")
    for k in data.keys():
        print(k, data[k])

mean_ps [-3.11092785 -3.73878997 -3.34517072 ... -9.81649124 -9.80565673
 -9.80594401]
components [[ 5.38518315e-03  3.29031591e-03 -2.73611289e-03 ... -1.18610207e-02
  -3.34105287e-03  8.90988988e-05]
 [ 3.45033122e-03  8.34700515e-07  3.72338291e-03 ... -1.84762558e-02
  -1.60027100e-02  2.05571576e-02]
 [ 1.39057944e-03 -1.06612430e-03  8.18225812e-04 ...  2.34760122e-03
  -5.90645576e-03  1.08358326e-02]
 ...
 [-1.57348407e-02 -4.25885729e-03 -4.25292595e-04 ... -8.20932383e-05
   6.67817973e-03  1.52282584e-02]
 [-2.55316721e-04  8.85253260e-03 -1.24878184e-04 ... -5.91773981e-03
  -7.13635044e-03  7.26510952e-03]
 [ 5.06069990e-03  2.04926651e-03 -5.81976522e-03 ...  1.06538025e-02
  -3.00308873e-02  2.80845359e-03]]
variance [4.10772728e-02 3.88451128e-02 3.70532747e-02 3.63494238e-02
 3.60586868e-02 3.50763280e-02 3.44229650e-02 3.41000404e-02
 3.39351202e-02 3.37949285e-02 3.33430047e-02 3.26348826e-02
 3.22883694e-02 3.18389471e-02 3.12708378e-02 3.08105055e-02
 3.00871345e-